In [1]:
import os
import pandas as pd
import numpy as np
from pydbmanager.connection import DatabaseConnection
from pydbmanager.operations import DatabaseOperations
from dotenv import load_dotenv
import warnings
warnings.filterwarnings('ignore')

## Database connection

In [2]:
# Load environment variables
load_dotenv()

# Initialize and test database connection
db = DatabaseConnection()
conn = db.create_connection()

if conn:
    print("\u2705 Connection Successful!")
    db.close_connection()
else:
    print("\u274c Connection Failed!")

2025-05-09 19:34:33,726 - INFO - Database connection established successfully.
2025-05-09 19:34:33,728 - INFO - Database connection closed


✅ Connection Successful!


In [3]:
db_ops = DatabaseOperations()

# testing the connection with a simple query
test = db_ops.query_data("SELECT top 10 * FROM patient_info", batch_size=5)
test.head()

2025-05-09 19:34:33,939 - INFO - Database connection established successfully.
2025-05-09 19:34:33,952 - INFO - Query executed in 0.0125 seconds


,patient_id,first_name,last_name,date_of_birth,gender,patient_type,external_id,patient_sub_type
0,5c96ba1a-8b2d-49bc-8e8e-b07761948286,Robin,Lopez,1973-05-23,female,caregiver,NULL,NULL
1,eda39327-b38f-41de-a46a-8782787369b7,Justin,Macks,1991-07-10,male,caregiver,NULL,NULL
2,6a7f7ce9-f63e-4651-b941-a25ce116de74,sandra,kinstle,1955-02-15,female,caregiver,NULL,NULL
3,096d402a-5fcd-41dd-b59b-b3089cf06742,Jing,Gu,1971-12-24,male,Stroke,NULL,PreviousStrokeUser
4,1715cc3c-bea6-4d7e-aad3-b2f96338ba60,Bootsie,Brenner,1958-01-28,female,Stroke,NULL,PreviousStrokeUser


## Data Loading

#### Patient Info ####

In [4]:
with open('../sql/patient_demo.sql', 'r') as file:
    patient_demo_sql = file.read()

In [5]:
patient_demo = db_ops.query_data(patient_demo_sql)
patient_demo.head()

2025-05-09 19:34:35,103 - INFO - Query executed in 0.1589 seconds


,patient_id,first_name,last_name,date_of_birth,postal_code,user_type,registered_at,country,referral_group,veteran,ethnicity,race,city,state,dark_mode,gender,patient_type,patient_sub_type,head_hit_count,has_tbi_before
0,0006ad41-c2d3-4994-8aab-7a3a107d50aa,CAROLINE,SUONG,1981-12-09,20866,patient,2022-06-08 16:37:57,US,BIA-GA,no,No,Korean,Burtonsville,MD,false,female,TBI/Concussion,tbiPatient,1.0,None
1,00469456-99a5-4c99-aa48-a918986c7c45,Lynsey,Alexander,1981-04-08,G66 8EG,patient,2021-05-19 15:27:06,GB,none,NULL,NULL,NULL,East Dunbartonshire,SCT,false,female,TBI/Concussion,tbiPatient,1.0,None
2,00469456-99a5-4c99-aa48-a918986c7c45,Lynsey,Alexander,1981-04-08,G66 8EG,patient,2021-05-19 21:12:51,GB,none,NULL,NULL,NULL,East Dunbartonshire,SCT,false,female,TBI/Concussion,tbiPatient,1.0,None
3,00e179cd-5edb-47fe-be53-b1e96e905433,Neil,Langrick,1963-11-30,WF178HZ,caregiver,2022-05-28 14:09:05,GB,NONE,notToAnswer,NULL,NULL,West Yorkshire,ENG,false,male,Other,Acquired Brain Injury,NaN,Yes
4,00e179cd-5edb-47fe-be53-b1e96e905433,Neil,Langrick,1963-11-30,WF17 8HZ,patient,2023-04-25 07:13:36,GB,none,no,notToAnswer,Prefer not to answer,zipcodeNotInLibrary,zipcodeNotInLibrary,false,male,Other,Acquired Brain Injury,NaN,Yes


In [6]:
patient_demo.shape

(1256, 20)

In [7]:
patient_demo.info()
# Check for missing values in the dataset       
missing_values = patient_demo.isnull().sum()
missing_values[missing_values > 0]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1256 entries, 0 to 1255
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   patient_id        1256 non-null   object        
 1   first_name        1256 non-null   object        
 2   last_name         1256 non-null   object        
 3   date_of_birth     1256 non-null   object        
 4   postal_code       1256 non-null   object        
 5   user_type         1256 non-null   object        
 6   registered_at     1256 non-null   datetime64[ns]
 7   country           1039 non-null   object        
 8   referral_group    935 non-null    object        
 9   veteran           1256 non-null   object        
 10  ethnicity         1256 non-null   object        
 11  race              1256 non-null   object        
 12  city              1241 non-null   object        
 13  state             1241 non-null   object        
 14  dark_mode         1256 n

country            217
referral_group     321
city                15
state               15
patient_type         6
head_hit_count     292
has_tbi_before    1187
dtype: int64

In [8]:
def check_missing_values(df):
    for col in df.columns:
        # Count string-based nulls (case-insensitive)
        string_nulls = df[col].astype(str).str.lower().isin(['null', 'none']).sum()
        
        if string_nulls > 0:
            print(f"Column '{col}' has {string_nulls} missing values (including 'NULL', 'None')")

In [9]:
check_missing_values(patient_demo)

Column 'country' has 220 missing values (including 'NULL', 'None')
Column 'referral_group' has 831 missing values (including 'NULL', 'None')
Column 'veteran' has 706 missing values (including 'NULL', 'None')
Column 'ethnicity' has 717 missing values (including 'NULL', 'None')
Column 'race' has 717 missing values (including 'NULL', 'None')
Column 'city' has 240 missing values (including 'NULL', 'None')
Column 'state' has 240 missing values (including 'NULL', 'None')
Column 'patient_type' has 6 missing values (including 'NULL', 'None')
Column 'patient_sub_type' has 86 missing values (including 'NULL', 'None')
Column 'has_tbi_before' has 1187 missing values (including 'NULL', 'None')


In [10]:
for col in patient_demo.columns:
    patient_demo[col] = (
        patient_demo[col]
        .astype(str)
        .str.strip()  
        .str.lower()
        .replace(['null', 'none', ''], np.nan)
    )

In [11]:
patient_demo['country'].fillna('Not Specified', inplace=True)
patient_demo['city'].fillna('Not Specified', inplace=True)
patient_demo['state'].fillna('Not Specified', inplace=True)

In [12]:
patient_demo['veteran'].fillna('No', inplace=True)
patient_demo['race'].fillna('Not Specified', inplace=True)
patient_demo['referral_group'].fillna('other', inplace=True)
patient_demo['ethnicity'].fillna('No', inplace=True)
patient_demo['patient_type'].fillna('Other', inplace=True)
patient_demo['patient_sub_type'].fillna('Not Specified', inplace=True)

In [22]:
# Convert 'nan' string to actual NaN
patient_demo['head_hit_count'] = patient_demo['head_hit_count'].replace('nan', np.nan)

# Now replace NaN with 0
patient_demo['head_hit_count'].fillna(0, inplace=True)

In [23]:
patient_demo.isna().sum()

patient_id             0
first_name             0
last_name              0
date_of_birth          0
postal_code            0
user_type              0
registered_at          0
country                0
referral_group         0
veteran                0
ethnicity              0
race                   0
city                   0
state                  0
dark_mode              0
gender                 0
patient_type           0
patient_sub_type       0
head_hit_count         0
has_tbi_before      1183
age                    0
dtype: int64

In [15]:
# calculating patient age from date of birth
patient_demo['date_of_birth'] = pd.to_datetime(patient_demo['date_of_birth'])
patient_demo['registered_at'] = pd.to_datetime(patient_demo['registered_at'])
# Calculate age in years    
patient_demo['age'] = (patient_demo['registered_at'] - patient_demo['date_of_birth']).dt.days // 365

In [16]:
# filtering out patients with age less than 10 or greater than 79
patient_demo = patient_demo[patient_demo['age'] > 10]

In [17]:
# filtering out the patients with user_type 4
patient_demo = patient_demo[patient_demo['user_type']!='4']

In [25]:
patient_demo['has_tbi_before'].fillna('Not Specified', inplace=True)

In [ ]:
# patient_demo.to_clipboard()

In [27]:
patient_demo.to_csv('../data/patient_demographic_data.csv', index=False)

#### TBI Incident

In [28]:
tbi_inc = db_ops.query_data("select * from tbi_incident")
tbi_inc.head()

2025-05-09 19:43:20,423 - INFO - Query executed in 0.0411 seconds


,id,tbi_incident_date,describe_event,injury_from,head_hit_location,patient_id,total_tbi,immediate_symptoms_resulting,num_head_hit_location
0,1080,2013-07-31,I was riding a longboard to my friends house w...,Fall,Left Side of Head,efd7682b-adde-4b6f-b2d7-16f50597c984,1,"Loss of Consciousness,Disorientation,Incoheren...",NULL
1,1164,2024-04-03,"On april 3, i was preforming my work duties . ...",Accident,Left Side of Head,bb8e0677-ddc8-49dd-be08-ab6f10ad43f3,1,"Disorientation,Confusion,Dazed or Vacant Stare",NULL
2,1105,2023-05-23,Drank a bottle of window cleaner with amonia.,Collision,"Front of Head,Back of Head",4471257f-7b52-4278-9654-e56cab633e67,5,"Loss of Consciousness,Disorientation,Confusion...",NULL
3,1051,2022-06-26,I was tubing on a lake behind a boat and got t...,Accident,"Front of Head,Right Side of Head,Back of Head,...",cfba5f4c-8266-4e4c-bcf8-36e423ab0e35,2,"Disorientation,Confusion,Dazed or Vacant Stare",NULL
4,1121,2023-07-11,Car Accident,Accident,Neck,fd6392ce-544f-4248-bf6a-9d9e454c8dfd,3,"Memory Loss,Sound sensitivity,Confusion,Disori...",NULL


In [29]:
tbi_inc.shape

(981, 9)

In [30]:
tbi_inc['tbi_incident_date'] = tbi_inc['tbi_incident_date'].astype(str).str.replace('1015-12-08', '2015-12-08')
tbi_inc['tbi_incident_date'] = pd.to_datetime(tbi_inc['tbi_incident_date'], errors='coerce')

In [31]:
for col in tbi_inc.columns:
    tbi_inc[col] = (
        tbi_inc[col]
        .astype(str)
        .str.strip()  
        .str.lower()
        .replace(['null', 'none', ''], np.nan)
    )

In [32]:
tbi_inc.isnull().sum()

id                                0
tbi_incident_date                 0
describe_event                    4
injury_from                      36
head_hit_location                72
patient_id                        0
total_tbi                         0
immediate_symptoms_resulting     53
num_head_hit_location           191
dtype: int64

In [33]:
tbi_inc['describe_event'].fillna('Not Specified', inplace=True)
tbi_inc['injury_from'].fillna('Not Specified', inplace=True)
tbi_inc['head_hit_location'].fillna('Not Specified', inplace=True)
tbi_inc['immediate_symptoms_resulting'].fillna('Not Specified', inplace=True)
tbi_inc['num_head_hit_location'].fillna('Not Specified', inplace=True)

In [34]:
tbi_inc.isna().sum()

id                              0
tbi_incident_date               0
describe_event                  0
injury_from                     0
head_hit_location               0
patient_id                      0
total_tbi                       0
immediate_symptoms_resulting    0
num_head_hit_location           0
dtype: int64

In [37]:
tbi_inc = tbi_inc[tbi_inc['total_tbi'].astype(int)<15]

In [39]:
tbi_inc.to_csv('../data/tbi_incident_data.csv', index=False)